<a href="https://colab.research.google.com/github/amalks02/finwise-genai-capstone/blob/chat-bot/chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U langchain-google-genai gradio

In [ ]:
# importing required packages
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import gradio as gr

In [ ]:
# Configure Gemini API
genai.configure(api_key="YOUR_GEMINI_API_KEY")

In [ ]:
# Define LLM
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

In [ ]:
# Adding Langchain memory
memory = ConversationBufferMemory(return_messages=True)

In [ ]:
# Customer data
customer_data = {
    "name": "Amit Sharma",
    "balance": 125000,
    "credit_card_due": 20000
}

In [ ]:
# loading customer profile into memory
memory.save_context(
    {"input": "System preload"},
    {"output": f"Customer profile: {customer_data}"}
)

In [ ]:
# Build conversational chain
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=False
)

In [ ]:
# function to process the user input
def process_user_input(user_input: str):
    global customer_data
    text = user_input.lower()

    # Balance inquiry
    if "balance" in text:
        return f"Your current balance is ₹{customer_data['balance']:,}."

    # Transfer money
    elif "transfer" in text:
        words = text.split()
        amount = None
        for word in words:
            if word.isdigit():
                amount = int(word)
                break

        if amount is None:
            return "Please tell me how much you want to transfer."

        if amount > customer_data["balance"]:
            return "Insufficient funds for this transfer."

        # Update balances
        customer_data["balance"] -= amount
        customer_data["credit_card_due"] = max(
            0, customer_data["credit_card_due"] - amount
        )

        return (f"₹{amount:,} transferred to your credit card. "
                f"New balance: ₹{customer_data['balance']:,}. "
                f"Remaining credit card due: ₹{customer_data['credit_card_due']:,}.")

    # Fallback to LLM
    else:
        return conversation.predict(input=user_input)


In [ ]:
# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## Banking Chatbot with Memory")

    chatbot = gr.Chatbot(height=400)
    msg = gr.Textbox(placeholder="Ask me about your account...")
    clear = gr.Button("Clear")

    def respond(user_input, chat_history):
        response = process_user_input(user_input)
        chat_history.append((user_input, response))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch()
